<a href="https://colab.research.google.com/github/Eugene31415/Pet-Projects/blob/main/%D0%A1%D0%B5%D0%BC%D0%B0%D0%BD%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B2%D0%B8%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings

С помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
100% 84.9M/84.9M [00:00<00:00, 183MB/s]
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

%matplotlib inline

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
dev_data["text"][0]

"@Claire_Nelson i'm on the north devon coast the next few weeks  will be down in Devon again in may sometime i hope though!"

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [ ]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text): # здесь text - это просто предложение или строка. Ну чисто один твит
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text.lower())
        filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        return filtered_tokens


    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]# Получи эмбеддинги слов и усредни их

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features'])

In [ ]:
examples["transformed_features"].shape

(1280, 2)

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Определим функции для тренировки и теста модели:

In [ ]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели
        optimizer.zero_grad()

        logits = model(features)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            logits = model(features)
            loss = criterion(logits, targets)
            acc = (logits.argmax(dim=1) == targets).type(torch.float).mean()

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, embeddings_dim, n_classes):
        super(NeuralNetwork, self).__init__()
        self.linear_1 = nn.Linear(embeddings_dim, 256)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(256, 128)
        self.linear_3 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.relu(self.linear_1(x))
        x = self.relu(self.linear_2(x))

        logits = self.linear_3(x)
        return logits

In [ ]:
import torch.nn as nn
from torch.optim import Adam

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-4
num_epochs = 2

model = NeuralNetwork(embeddings_dim=vector_size, n_classes=num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5020828288793564, 'Test Acc': 0.75190625}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.49676597118377686, 'Test Acc': 0.7558984375}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.49806737052365996, 'Test Acc': 0.7553102535942492}


## Embeddings for unknown words
---
Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. 
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. 

In [ ]:
class Custom_TwitterDataset_v1(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, window_size=1):
        super(Custom_TwitterDataset_v1, self).__init__(data, feature_column, target_column, word2vec)
        self.window_size = window_size

    def get_embeddings_(self, tokens):
        embeddings = []

        for i, word in enumerate(tokens):
            if word not in self.word2vec: # если слова нет среди предобученных эмбеддингов, расчитываем его эмбеддинг по его контенту
                left_context = max(0, i - self.window_size)
                right_context = min(len(tokens) - 1, i + self.window_size)

                context = tokens[left_context: i] + tokens[i + 1: right_context]
                embedding = [(self.word2vec.get_vector(context_word) - self.mean) / self.std for context_word in context if context_word in self.word2vec]
                
                if not len(embedding):
                    embedding = np.zeros(self.word2vec.vector_size)
                else:
                    embedding = np.sum(embedding, axis=0)

                embeddings.append(embedding)
            else:
                embeddings.append((self.word2vec.get_vector(word) - self.mean) / self.std)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
custom_dev_v1 = Custom_TwitterDataset_v1(dev_data, "text", "emotion", word2vec)

In [ ]:
train_v1, valid_v1 = random_split(custom_dev_v1, [train_size, len(custom_dev_v1) - train_size])

train_loader_v1 = DataLoader(train_v1, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_v1 = DataLoader(valid_v1, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader_v1, e, device)
    log = testing(model, criterion, valid_loader_v1, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_v1.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.49004132604599, 'Test Acc': 0.759890625}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.48866060304641723, 'Test Acc': 0.7607578125}


In [ ]:
test_loader_v1 = DataLoader(
    Custom_TwitterDataset_v1(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_v1.pt", map_location=device))

print(testing(model, criterion, test_loader_v1, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.4938968542856149, 'Test Acc': 0.757232178514377}


In [ ]:
!pip install sparsesvd

  Created wheel for sparsesvd: filename=sparsesvd-0.2.2-cp37-cp37m-linux_x86_64.whl size=295922 sha256=fe5c5093e7e2cc3de2cc2f43005a2d77a725d1ddf5b8874b204e09355b714d89
  Stored in directory: /root/.cache/pip/wheels/4f/e5/2e/50014c1a0983cb8a0738d8c672ef890ef29262779c0259f1e3
Successfully built sparsesvd


In [ ]:
# Будем использовать sparsesvd разложение, так как размер дефолтного ембеддинга (предобученного) - 300, а получившегося после TF - IDF размер вектора ембеддинга V 
# то есть кол-во документов, которое около 1600000. Поэтому sparsesvd позволяет задать размер диагональной матрицы, а соответсвенно перейти к матриче меньшей размерности.

In [ ]:
from sparsesvd import sparsesvd
from sklearn.feature_extraction.text import TfidfVectorizer

def get_tfidf_embeddings(corpus_docs, embedding_size=300):
    tfidf_vectorizer = TfidfVectorizer(min_df=5)
    sparse_matrix_tfidf = tfidf_vectorizer.fit_transform(corpus_docs)
    
    u, s, vt = sparsesvd(sparse_matrix_tfidf.tocsc(), embedding_size)
    k, v = tfidf_vectorizer.get_feature_names(), vt.T

    return {k[i]: v[i] for i in range(len(v))}

In [ ]:
tfidf_embeddings = get_tfidf_embeddings(corpus_docs=data["text"].values, embedding_size=300)

In [ ]:
class Custom_TwitterDataset_v2(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        super(Custom_TwitterDataset_v2, self).__init__(data, feature_column, target_column, word2vec)

        self.tfidf_embeddings = tfidf_embeddings

    def get_embeddings_(self, tokens):
        embeddings = []

        for i, word in enumerate(tokens):
            if word not in self.word2vec:
                if word in self.tfidf_embeddings:
                    embeddings.append((self.tfidf_embeddings[word] - self.mean) / self.std)
                else:
                    pass
            else:
                if word in self.tfidf_embeddings:
                    embeddings.append((self.word2vec.get_vector(word) + self.tfidf_embeddings[word] - self.mean) / self.std)
                else:
                    embeddings.append((self.word2vec.get_vector(word) - self.mean) / self.std)

                    
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
custom_dev_v2 = Custom_TwitterDataset_v2(dev_data, "text", "emotion", word2vec)

In [ ]:
train_v2, valid_v2 = random_split(custom_dev_v2, [train_size, len(custom_dev_v2) - train_size])

train_loader_v2 = DataLoader(train_v2, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_v2 = DataLoader(valid_v2, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader_v2, e, device)
    log = testing(model, criterion, valid_loader_v2, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_v2.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5041082627773285, 'Test Acc': 0.74996484375}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.49802030229568484, 'Test Acc': 0.75430078125}


In [ ]:
test_loader_v2 = DataLoader(
    Custom_TwitterDataset_v2(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_v2.pt", map_location=device))

print(testing(model, criterion, test_loader_v2, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.4953616586165687, 'Test Acc': 0.7564490565095847}


In [ ]:
# Глядя на получившиеся accuracy нельзя с большой точностью сказать, какой из подходов лучше. Это будет не совсем корректно. Идея, учитывающая контекстные слова
# для центральных слов, чьих ембеддингов у нас нет, очень интересная сама по себе, но и идея совместить и потом усреднить TF - IDF ембеддинги и предобученные тоже дает хорошие
# результаты (на нашем локальном эксперименте даже лучше, чем первый подход). Я бы совместил оба подхода :)
# 
